In [1]:
import pandas as pd
import json
import ast

### Loading raw csv file into dataframe and checking characteristics

In [5]:
keywords = pd.read_csv("data/raw/keywords.csv")
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


### Important note: 
The file consists of two columns containing JSON objects. However, the JSON is not properly formatted. Therefore it is required to perform transformations to enable unpacking the data and loading it into database.

As a result, credits.csv has to be divided into two seperate files/tables, representing columns 'cast' and 'crew'.

### 1. Parsing "keywords" column

In [8]:
keywords_list = []

for entry in keywords['keywords']:
    if pd.notna(entry):
        try:
            keywords_data = ast.literal_eval(entry)
            keywords_list.append(keywords_data)
        except (SyntaxError, ValueError):
            keywords_list.append([])
    else:
        keywords_list.append([])

keywords['keywords'] = keywords_list

We received a list with cast, but still it is needed to link the data with movie id. For this purpose the index will be used.

In [9]:
keywords_df = pd.DataFrame()
for i, item in enumerate(keywords_list):
    df = pd.DataFrame(keywords_list[i])
    df["index"] = i
    keywords_df = pd.concat([keywords_df, df], axis=0, ignore_index=True)

keywords_df.head()

,id,name,index
0,931.0,jealousy,0
1,4290.0,toy,0
2,5202.0,boy,0
3,6054.0,friendship,0
4,9713.0,friends,0


In [11]:
keywords_df.to_csv('data/ready_for_ingestion/keywords.csv', index=False)

In [25]:
keywords_df = pd.read_csv('data/ready_for_ingestion/keywords.csv')
keywords_df

,id,name,index
0,931.0,jealousy,0
1,4290.0,toy,0
2,5202.0,boy,0
3,6054.0,friendship,0
4,9713.0,friends,0
...,...,...,...
158675,11800.0,mockumentary,46411
158676,10703.0,tragic love,46414
158677,2679.0,artist,46415
158678,14531.0,play,46415


In [26]:
movies_ids = keywords[["id"]].reset_index()
movies_ids

,index,id
0,0,862
1,1,8844
2,2,15602
3,3,31357
4,4,11862
...,...,...
46414,46414,439050
46415,46415,111109
46416,46416,67758
46417,46417,227506


In [27]:
#movies_ids = keywords[["movie_id"]].reset_index()
keywords_df = pd.merge(movies_ids, keywords_df, how="left", on="index")
keywords_df

,index,id_x,id_y,name
0,0,862,931.0,jealousy
1,0,862,4290.0,toy
2,0,862,5202.0,boy
3,0,862,6054.0,friendship
4,0,862,9713.0,friends
...,...,...,...,...
173470,46415,111109,14531.0,play
173471,46415,111109,215397.0,pinoy
173472,46416,67758,NaN,NaN
173473,46417,227506,NaN,NaN


In [28]:
keywords_df = keywords_df.rename(columns={'id_y': 'keyword_id', 'id_x': 'movie_id'})
keywords_df

,index,movie_id,keyword_id,name
0,0,862,931.0,jealousy
1,0,862,4290.0,toy
2,0,862,5202.0,boy
3,0,862,6054.0,friendship
4,0,862,9713.0,friends
...,...,...,...,...
173470,46415,111109,14531.0,play
173471,46415,111109,215397.0,pinoy
173472,46416,67758,NaN,NaN
173473,46417,227506,NaN,NaN


Since Index column was used only to perform join operation, it is not needed anymore.

In [29]:
keywords_df = keywords_df.drop('index', axis=1)
keywords_df

,movie_id,keyword_id,name
0,862,931.0,jealousy
1,862,4290.0,toy
2,862,5202.0,boy
3,862,6054.0,friendship
4,862,9713.0,friends
...,...,...,...
173470,111109,14531.0,play
173471,111109,215397.0,pinoy
173472,67758,NaN,NaN
173473,227506,NaN,NaN


### Saving the output to dedicated catalog

In [30]:
keywords_df.to_csv('data/ready_for_ingestion/keywords.csv', index=False)